<h2><font color="#004D7F" size=6>Módulo 6. Fase de optimización y forecasting</font></h2>



<h1><font color="#004D7F" size=5>2. Procesamiento de datos avanzado</font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Machine Learning con Python</font><br>

---

<h2><font color="#004D7F" size=5>Índice</font></h2>
<a id="indice"></a>

* [1. Introducción](#section1)
* [2. Valores Perdidos](#section2)
* [3. Escalar el atributo clase](#section3)
* [4. One-Hot Encoding](#section4)

In [1]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98% }</style>"))

C:\Users\eel20\AppData\Local\Temp\ipykernel_3364\1126351303.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

Los transformadores son aquellos algoritmos y funciones que toman como datos de entrada las variables de nuestro problema y devuelven otras variables.

Estas transformaciones suelen __aplicarse antes de un algoritmo de clasificación/regresión__, y todas aquellas operaciones que se ejecutan antes de un algoritmo de predicción reciben el nombre de __preprocesamiento__.

En esta sección, aunque corresponda a la fase de procesamiento de datos, vamos a trabajar aspectos avanzados en el procesamiento de datos. Estos casos a estudiar son casos que nos vamos a encontrar muchísimo cuando trabajamos un proyecto de machine learning, además vamos a ver los resultados que nos dan los algoritmos para ver su mejora.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section2"></a> 
# <font color="#004D7F">2. Valores perdidos</font>

Una de las transformaciones más básicas consiste en la sustitución de valores perdidos o `NaN's` de la base de datos, ya que muchos algoritmos no son capaces de manejarlos. Una de las formas de realizar esta sustitución de valores perdidos consiste en utilizar la __media (para valores continuos) o la moda (caso discreto)__ con los casos conocidos y aplicar dicha métrica a los `NaN's` de la variable.

La función `sklearn.impute.SimpleImputer` se encarga de calcular y modificar los datos de entrada. Vamos a ver cómo se utiliza en el caso de Wisconsin.

<div class="alert alert-block alert-info">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Documentación oficial de la clase [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer).
</div> 

<a id="section21"></a> 
## <font color="#004D7F">2.1. Caso genérico</font>

Lo primero que vamos a realizar es ver que atributos tienen valores perdidos, para ello vamos a utilizar las funciones de NumPy que nos permite analizarlos.

In [2]:
import numpy as np
import pandas as pd
wisconsin = pd.read_csv('data/wisconsin.csv', dtype={ "label": 'category'})
wisconsin
# Lo primero es separar los atributos de la clase. 1 es por la columna
wisconsin_data = wisconsin.drop('label', 1)
wisconsin_target = wisconsin['label']

# Después comprobamos los NaN's de nuestros datos, esto podemos hacerlo con Numpy.
print(np.sum(np.isnan(wisconsin_data)))

# Vemos que BareNuclei es el único que tiene 32 valores


patientId            0
clumpThickness       0
cellSize             0
CellShape            0
marginalAdhesion     0
epithelialSize       0
bareNuclei          32
blandChromatin       0
normalNucleoli       0
mitoses              0
dtype: int64


C:\Users\eel20\AppData\Local\Temp\ipykernel_3364\1322922485.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  wisconsin_data = wisconsin.drop('label', 1)


Solo la variable `bareNuclei` tiene valores perdidos, por lo que vamos a aplicar el `SimpleImputer` y ver que resultados se obtienen.

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Importante: Debido a los `NaNs` el algoritmo no podrá ejecutarse, por lo que es necesario utilizar alguna de las bases de datos `wisconsin` a las que se les ha aplicado el `Imputer` en la sección anterior o volver a aplicarlo ahora.
</div>

In [3]:
# Importamos la clase LinearRegression
from sklearn.linear_model import LogisticRegression

# creamos un objeto con los parámetros por defecto
model = LogisticRegression()
# entrenamos con los datos de entrada y la salida
model.fit(wisconsin_data, wisconsin_target)
# Este algoritmo no trabaja con NaN

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

En este caso, los valores perdidos han sido sustituidos por la media de los valores conocidos en `bareNuclei`. Si vamos a la documentación del [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer), podremos observar los diferentes parámetros de configuración del algoritmo. Si queremos modificar el comportamiento del algoritmo deberemos definir estos parámetros en el constructor cuando creemos el objeto. 

In [3]:
from sklearn.impute import SimpleImputer

# Creamos un objeto de la clase Imputer con los parámetros por defecto
imp = SimpleImputer(strategy='median')

# Llamamos a fit, para que aprenda la media con la que tiene que rellenar los NaN's y le pasamos los datos
imp.fit(wisconsin_data)

# Finalmente, transformamos los datos con las medias aprendidas.
wisconsin_trans = imp.transform(wisconsin_data)

# Aunque le hayamos pasado un pandas a SciKit, este nos devuelve un numpy array, por lo que volvemos a 
# trasnformarlo a pandas
wisconsin_trans = pd.DataFrame(wisconsin_trans, columns = wisconsin_data.columns)
wisconsin_trans

# Si volvemos a comprobar los NaN's en la nueva base de datos, vemos que ya no hay
print(np.sum(np.isnan(wisconsin_trans)))

patientId           0
clumpThickness      0
cellSize            0
CellShape           0
marginalAdhesion    0
epithelialSize      0
bareNuclei          0
blandChromatin      0
normalNucleoli      0
mitoses             0
dtype: int64


Ahora si que podemos lanzar el algoritmo ya que no tenemos ningún valor NaN en nuestro conjunto de datos.

In [4]:
# Importamos la clase LinearRegression
from sklearn.linear_model import LogisticRegression

# creamos un objeto con los parámetros por defecto
model = LogisticRegression(solver = 'lbfgs', max_iter=1000)

# entrenamos con los datos de entrada y la salida
model.fit(wisconsin_trans, wisconsin_target)

# obtenemos una predicción para los datos de wisconsin
model_pred = model.predict(wisconsin_trans)
score = model.score(wisconsin_trans, wisconsin_target)

# Otenemos el Accuracy
print(f"Accuracy: {score}")

Accuracy: 0.9699570815450643


<a id="section22"></a>
## <font color="#004D7F"> 2.2. Pipeline con `SimpleImputer` </font>

En el este ejemplo vamos a definir un Pipeline con algunas de las clases que hemos utilizado. Vamos a utilizar `wisconsin_data`, que si recordamos, tenía ciertos valores perdidos, por lo que no podía ser pasado directamente al clasificador. Por eso, vamos a crear un Pipeline que primero utilice un `SimpleImputer` y luego llame a `LogisticRegression`.

Igual que en todos los casos anteriores, utilizaremos el Pipeline con las funciones `fit`, `predict` y `score`, de la misma forma que si se tratase de un clasificador. Para crear un Pipeline se utiliza una lista de tuplas (clave, valor) donde la clave es un string representativo y el valor es el estimador.

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe = Pipeline([('Imp', SimpleImputer()),
                ('LoR', LogisticRegression(solver='lbfgs', max_iter=1000))])

pipe.fit(wisconsin_data, wisconsin_target)
print(f"Tasa de acierto: {pipe.score(wisconsin_data, wisconsin_target)}")




Tasa de acierto: 0.9699570815450643


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section3"></a>
# <font color="#004D7F"> 3. Escalar atributo clase(Target)</font>

En problemas de modelado predictivo de regresión donde se debe predecir un valor numérico, también puede ser crítico escalar y realizar otras transformaciones de datos en la variable objetivo. Esto se puede lograr en Python usando la clase `TransformedTargetRegressor`. Para problemas de regresión, a menudo es deseable escalar o transformar tanto las variables de entrada como las de destino

<div class="alert alert-block alert-info">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Documentación oficial de la clase [`TransformedTargetRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html?highlight=transformedtargetregressor#sklearn.compose.TransformedTargetRegressor).
</div> 

In [6]:
# example of normalizing input and output variables for regression.
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
filename = 'data/housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataframe = pd.read_csv(filename, delim_whitespace=True, names=names) 
array = dataframe.values
#X las Caracteristicas
X = array[:,0:13]
#Y son los Target (clases)
Y = array[:,13]

In [7]:
# prepare the model with input scaling
## en este conjunto de datos aplicara (una transformacion MinMaxScaler a las caracteristicas) y luego
## applicar un algoritmo de machine learning (HuberRegressor) 
pipeline = Pipeline(steps = [('Nomalize', MinMaxScaler()), ('HuR', HuberRegressor())])

# prepare the model with target scaling
## para utilizasr el pipeline donde aplicara las caracteristicas y ademas
## aplicara una transformacion en los dartos con MinMaxScaler
model = TransformedTargetRegressor(regressor=pipeline, transformer= MinMaxScaler())

# evaluate model
cv =KFold(n_splits=10, shuffle=True, random_state=7)
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv)

# convert scores to positive
## hace que el resultado final nunca sea negativo
scores = np.absolute(scores)

# summarize the result
s_mean = np.mean(scores)
print(f"MAE: {s_mean}")

## el resultado en este caso MAE: 3.2095842117833655 es el error medio 

MAE: 3.2095842117833655


Si, por ejemplo ahora le hacemos una transformación de Yeo-Johnson vemos como mejoran nuestros resultados.

In [9]:
from sklearn.preprocessing import PowerTransformer
# prepare the model with input scaling
???
# prepare the model with target scaling
???
# evaluate model
???
# convert scores to positive
???
# summarize the result
???

Mean MAE: 2.926


<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<a id="section4"></a>
# <font color="#004D7F"> 4. One-Hot Encoding</font>


Aplicar transformaciones de datos como escalar o codificar variables categóricas es sencillo cuando todas las variables de entrada son del mismo tipo. Puede ser un desafío cuando tiene un conjunto de datos con tipos mixtos y desea aplicar transformaciones de datos selectivamente a algunas, pero no a todas, las características de entrada.

Afortunadamente, la biblioteca de aprendizaje automático Python scikit-learn proporciona el `ColumnTransformer` que le permite aplicar transformaciones de datos de forma selectiva a diferentes columnas de su conjunto de datos.

Para este ejemplo vamos a utilizar el conjunto de datos [Abalone](https://archive.ics.uci.edu/ml/datasets/Abalone) el cual es un problema de clasificación pero tiene características numéricas y categóricas que tenemos que transformar para que todas se encuentren en un mismo tipo. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Documentación oficial de la clase [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html?highlight=columntransformer#sklearn.compose.ColumnTransformer).
</div> 

In [18]:
# example of using the ColumnTransformer for the Abalone dataset
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
# load dataset
filename = 'data/abalone.csv'
dataframe = pd.read_csv(filename, header=None) 
array = dataframe.values
# split into inputs and outputs
???
# determine categorical and numerical features
???
# define the data preparation for the columns
???
# define the model
???
# define the data preparation and modeling pipeline
???
# define the model cross-validation configuration
???
# evaluate the pipeline using cross validation and calculate MAE
???
# convert MAE scores to positive values
???
# summarize the model performance
???

MAE: 1.465 (0.047)


En este caso, logramos un MAE promedio de aproximadamente 1.4, que es mejor que el puntaje inicial de 2.3. Es por eso que es muy importante tener todos los atributos del mismo tipo, porque los algoritmos, sobre todo los basados en funciones o representaciones en funciones mejoran considerablemente.

Los algoritmos tipo árboles tienen un comportamiento contrario, es decir, prefieren que los atributos sean tipo categóricos aunque su desempeño no mejora tanto como los basados en funciones a pasar a numérico.

<div style="text-align: right"> <font size=5>
    <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a>
</font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>